In [1]:
# From predict.py
# Standard library imports
import os
import time
from glob import glob
from pathlib import Path

# Third party imports
import numpy as np
from tensorflow import keras

from ..georeferencing import georeference
from ..utils import remove_files
from .utils import open_images, save_mask

BATCH_SIZE = 8
IMAGE_SIZE = 256
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

ImportError: attempted relative import with no known parent package

In [10]:
# Model import
# model = keras.models.load_model(checkpoint_path)
import tensorflow as tf
class F1_Score(keras.metrics.Metric):
    #  from https://stackoverflow.com/a/64477522
    def __init__(self, class_id, name='f1_score', **kwargs):
        super().__init__(name=name, **kwargs)
        self.f1 = self.add_weight(name='f1', initializer='zeros')
        self.precision_fn = tf.keras.metrics.Precision(class_id=class_id)
        self.recall_fn = tf.keras.metrics.Recall(class_id=class_id)

    def update_state(self, y_true, y_pred, sample_weight=None):
        p = self.precision_fn(y_true, y_pred)
        r = self.recall_fn(y_true, y_pred)
        self.f1.assign(tf.math.divide_no_nan(2 * (p * r), (p + r)))

    def result(self):
        return self.f1

    def reset_states(self):
        # we also need to reset the state of the precision and recall objects
        self.precision_fn.reset_states()
        self.recall_fn.reset_states()
        self.f1.assign(0)

# def get_f1_score_fn(cfg):
#     return F1_Score(class_id=1)


checkpoint_path = "/home/annazan/fAIr-utilities/ramp-data/metric_data/model97_td372/train/model-checkpts/20240514-214549/model_20240514-214549_020_0.970.tf"
model = keras.models.load_model(checkpoint_path,
                                custom_objects={"F1_Score":F1_Score(class_id=1)},
                                compile=False)
model.summary()

TypeError: __init__() missing 1 required positional argument: 'class_id'